# 一、功能探索

In [1]:
import os
import torch
import transformers

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\GXL\.conda\envs\huggingface\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GXL\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:

# token_ids = tokenizer.encode("Hello, world!")
token_ids = tokenizer.encode("Hello, world!", add_special_tokens=False)
token_ids
original_text = tokenizer.decode(token_ids)
tokens = tokenizer.convert_ids_to_tokens(token_ids)
token_ids, original_text, tokens

([7592, 1010, 2088, 999], 'hello, world!', ['hello', ',', 'world', '!'])

In [13]:
encoded_text

{'input_ids': tensor([[ 101, 7592, 1010, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [5]:
torch.cuda.is_available()

False

In [5]:
os.environ['http_proxy'] = 'http://127.0.0.1:33210'
os.environ['https_proxy'] = 'http://127.0.0.1:33210'
os.environ['all_proxy'] = 'socks5://127.0.0.1:33211'

In [8]:
# 使用os配置代理就能成功！
# 不知道后面下载模型和上传模型会不会顺利
from huggingface_hub import notebook_login

notebook_login()

In [37]:
# 加载tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\GXL\.conda\envs\huggingface\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GXL\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## 对tokenizer功能的探索

其中涉及三种状态：

- 文本text
    - 字符串
    - [字符串1, 字符串2, .. 字符串n]
- 分本分词后的tokens
    - [token1, token2, ... , token_n]
    - 其中可能包含填充的特殊标记[CLS][SEP]
    - 也可能不包括（直接使用encode方法）
- 分词并转化为digit的input_ids
    - 只能是字典，包括"input_ids","attention_mask"这些键，值是list

<img src="..\Apictures\11.png" alt="11" style="zoom:85%;" />

    - 两个convert方法的输入和输出都是list
  
    - 通过encode_plus()和__call__方法得到的输出是字典，而encode()得到的输出只是一个digit的list，
      但是三者的digit中都带有特殊token对应的digit(默认带，也可以通过设置add_special_tokens=False让它不带）
      
    - tokenize()方法产生的tokens中不带有特殊的token([CLS][SEP]这些
    
    - 对于list(str)的输入：
        __call__方法产生的字典中"input_ids"对应的digit是多维的，相当于多个句子分别编码
        encode()和encode_plus()只能接收list中有两个str,表示一个句子对，产生的结果是讲这两个句子对拼接成一个句子，而不是两个句子单独处理
        两个encode方法在输入上也有一些微小的差别，encode_plus()还可以接收两个sentence的str传入作为句子对，
        而encode只能接收list(str)作为句子对

In [ ]:
#text = "Hello, Hugging Face!"
# text2 = "oh yeah!"
# text = ["Hello, Hugging Face!", "Transformers are amazing.", "good example！"]
text = ["Hello, Hugging Face!", "Transformers are amazing."]


# 文本 → input（其中还包括了attention_mask）
# 底层调用的是__call__方法
encoded_input = tokenizer(text)
print("encoded_input:\n" + str(encoded_input))

# encode与__call__进行对比
encode_input = tokenizer.encode(text)
print("encode_input:\n" + str(encode_input))
# encode_plus与__call__对比
encode_plus_input = tokenizer.encode_plus(text)
print("encode_plus_input:\n" + str(encode_plus_input))

tokens = tokenizer.tokenize(text)
print("tokens:\n" + str(tokens))

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("tokens_ids:\n" + str(tokens_ids))
tokens_ids = tokenizer(tokens, is_split_into_words=True)
print("tokens_ids:\n" + str(tokens_ids))

tokens_reverse = tokenizer.convert_ids_to_tokens(encode_input)
tokens_reverse, tokenizer.convert_tokens_to_ids(tokens_reverse)

## 数据集预处理

In [6]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [288]:
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [135]:
wnut['train'], wnut["train"].features, type(wnut["train"].features)

(Dataset({
     features: ['id', 'tokens', 'ner_tags'],
     num_rows: 3394
 }),
 {'id': Value(dtype='string', id=None),
  'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)},
 datasets.features.features.Features)

In [130]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [131]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [133]:
tokenized_wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1287
    })
})

In [132]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [115]:
texts = [["how", "are", "you?", "?"], ["I'm very very fxxking", "fine", "thanks", "!", "and", "you?"]]
# texts = ["how are you?", "I am fine, and you?"]

tokenized_inputs = tokenizer(texts, truncation=True, is_split_into_words=True)
print("tokenized_inputs:\n" + str(tokenized_inputs))
tokenized_inputs.word_ids 

tokenized_inputs:
{'input_ids': [[101, 2129, 2024, 2017, 1029, 1029, 102], [101, 1045, 1005, 1049, 2200, 2200, 23292, 2595, 6834, 2986, 4283, 999, 1998, 2017, 1029, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


<bound method BatchEncoding.word_ids of {'input_ids': [[101, 2129, 2024, 2017, 1029, 1029, 102], [101, 1045, 1005, 1049, 2200, 2200, 23292, 2595, 6834, 2986, 4283, 999, 1998, 2017, 1029, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}>

In [129]:
examples = wnut['train'][:3]
print("examples:\n" + str(examples) + "\n")

tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
print("tokenized_inputs:\n" + str(tokenized_inputs) + '\n')

# word_ids是一个方法，不指定batch_index至少加上括号啊！（不指定默认是0）
print(tokenized_inputs.word_ids(batch_index=1))


examples:
{'id': ['0', '1', '2'], 'tokens': [['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.'], ['From', 'Green', 'Newsfeed', ':', 'AHFA', 'extends', 'deadline', 'for', 'Sage', 'Award', 'to', 'Nov', '.', '5', 'http://tinyurl.com/24agj38'], ['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']], 'ner_tags': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

tokenized_inputs:
{'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024

In [88]:
# 进一步分词为subwords

example = wnut["train"][0]
print("example:\n" + str(example) + "\n")

tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
tokenized_inputs

example:
{'id': '0', 'tokens': ['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]}



{'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
label_list = ["O", "B-BIAS", "I-BIAS"] # 根据需要添加其他标签
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}


# 二、训练Bias-NER模型

## 1.数据预处理

In [144]:
# 加载数据
bias_conll = load_dataset("newsmediabias/BIAS-CONLL")

C:\Users\GXL\.conda\envs\huggingface\lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [159]:
bias_conll, bias_conll["train"][0:3]

(DatasetDict({
     train: Dataset({
         features: ['Unnamed: 0', 'sentence_id', 'words', 'labels'],
         num_rows: 50575
     })
 }),
 {'Unnamed: 0': [0, 1, 2],
  'sentence_id': [0, 0, 0],
  'words': ['rachel_scanlan', 'thanks', 'for'],
  'labels': ['O', 'O', 'O']})

In [160]:
# 定义分组和聚合函数
def group_words_labels(examples):
    # 分组操作在这里是逐个执行的，所以我们需要先将它们累积在列表中
    grouped_words = []
    grouped_labels = []
    previous_id = None
    words = []
    labels = []
    for i in range(len(examples['sentence_id'])):
        if previous_id is None or previous_id == examples['sentence_id'][i]:
            # 如果是同一句子，累积单词和标签
            words.append(examples['words'][i])
            labels.append(examples['labels'][i])
        else:
            # 如果是新句子，保存前一句子的累积结果，并重新开始累积
            grouped_words.append(words)
            grouped_labels.append(labels)
            words = [examples['words'][i]]
            labels = [examples['labels'][i]]
        previous_id = examples['sentence_id'][i]
        if(i % 1000 == 0):
            print("第" + str(i) + "个样本已经处理完")
    # 确保最后一句话也被添加
    grouped_words.append(words)
    grouped_labels.append(labels)
    # 返回分组结果
    return {'words': grouped_words, 'labels': grouped_labels}

# 对训练集应用分组函数
grouped_train_dataset = group_words_labels(bias_conll["train"])

第0个样本已经处理完
第1000个样本已经处理完
第2000个样本已经处理完
第3000个样本已经处理完
第4000个样本已经处理完
第5000个样本已经处理完
第6000个样本已经处理完
第7000个样本已经处理完
第8000个样本已经处理完
第9000个样本已经处理完
第10000个样本已经处理完
第11000个样本已经处理完
第12000个样本已经处理完
第13000个样本已经处理完
第14000个样本已经处理完
第15000个样本已经处理完
第16000个样本已经处理完
第17000个样本已经处理完
第18000个样本已经处理完
第19000个样本已经处理完
第20000个样本已经处理完
第21000个样本已经处理完
第22000个样本已经处理完
第23000个样本已经处理完
第24000个样本已经处理完
第25000个样本已经处理完
第26000个样本已经处理完
第27000个样本已经处理完
第28000个样本已经处理完
第29000个样本已经处理完
第30000个样本已经处理完
第31000个样本已经处理完
第32000个样本已经处理完
第33000个样本已经处理完
第34000个样本已经处理完
第35000个样本已经处理完
第36000个样本已经处理完
第37000个样本已经处理完
第38000个样本已经处理完
第39000个样本已经处理完
第40000个样本已经处理完
第41000个样本已经处理完
第42000个样本已经处理完
第43000个样本已经处理完
第44000个样本已经处理完
第45000个样本已经处理完
第46000个样本已经处理完
第47000个样本已经处理完
第48000个样本已经处理完
第49000个样本已经处理完
第50000个样本已经处理完


NameError: name 'Dataset' is not defined

In [187]:
# 映射

label_list = ['O', 'B-BIAS', 'I-BIAS'] # 根据需要添加其他标签
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

In [194]:
# 将labels从类标签转化为digits

# 转换标签到 ID 的函数
def labels_to_ids(examples):
    # 使用标签到 ID 的映射来转换每个标签
    try:
        examples['labels'] = [[label_to_id[label] for label in sentence_labels] for sentence_labels in examples['labels']]
    except KeyError as e:
        print(f"找不到标签: {e}")
    return examples

# 在 grouped_train_dataset 上应用这个函数
grouped_train_dataset = grouped_train_dataset.map(labels_to_ids, batched=True)

# 查看转换后的前两个样本
print(grouped_train_dataset[0:2])


Map:   0%|          | 0/2715 [00:00<?, ? examples/s]

{'words': [['rachel_scanlan', 'thanks', 'for', 'following', 'your', 'website', 'looks', 'like', 'greek', 'to', 'me', 'and', 'i', 'couldn', 't', 'see', 'your', 'tweets'], ['Waiting', 'for', 'it', 'to', 'rain', 'for', 'our', 'golf', 'tournament', 'today']], 'labels': [[0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 2, 0, 0, 1, 0, 0, 0, 0], [1, 2, 0, 0, 0, 0, 0, 1, 2, 0]]}


In [195]:
from datasets import Dataset

# 将聚合后的数据转换为 Dataset 类型
grouped_train_dataset = Dataset.from_dict({
    'words': grouped_train_dataset['words'],
    'labels': grouped_train_dataset['labels']
})

# 显示结果
grouped_train_dataset, grouped_train_dataset[0: 2]


(Dataset({
     features: ['words', 'labels'],
     num_rows: 2715
 }),
 {'words': [['rachel_scanlan',
    'thanks',
    'for',
    'following',
    'your',
    'website',
    'looks',
    'like',
    'greek',
    'to',
    'me',
    'and',
    'i',
    'couldn',
    't',
    'see',
    'your',
    'tweets'],
   ['Waiting',
    'for',
    'it',
    'to',
    'rain',
    'for',
    'our',
    'golf',
    'tournament',
    'today']],
  'labels': [[0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 2, 0, 0, 1, 0, 0, 0, 0],
   [1, 2, 0, 0, 0, 0, 0, 1, 2, 0]]})

## 2.加载tokenizer,进一步处理数据

### 2.1 加载tokenizer

In [170]:
# 前面已经加载过了，所以这里只是粘贴一下代码

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

### 2.2 添加id列  & 修改words列和labels列名字

In [295]:
id_list = [str(i) for i in range(len(grouped_train_dataset))]

# 必须添加id列
# 因为map()方法进行分批时，需要用到id,如果没有就会报错
# grouped_train_dataset = grouped_train_dataset.add_column("id", id_list)


# 给words和labels列改名
# grouped_train_dataset = grouped_train_dataset.rename_column("labels", "ner_tags")
# grouped_train_dataset = grouped_train_dataset.rename_column("words", "tokens")
grouped_train_dataset, grouped_train_dataset[0:2] 

(Dataset({
     features: ['tokens', 'ner_tags', 'id'],
     num_rows: 2715
 }),
 {'tokens': [['rachel_scanlan',
    'thanks',
    'for',
    'following',
    'your',
    'website',
    'looks',
    'like',
    'greek',
    'to',
    'me',
    'and',
    'i',
    'couldn',
    't',
    'see',
    'your',
    'tweets'],
   ['Waiting',
    'for',
    'it',
    'to',
    'rain',
    'for',
    'our',
    'golf',
    'tournament',
    'today']],
  'ner_tags': [[0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 2, 0, 0, 1, 0, 0, 0, 0],
   [1, 2, 0, 0, 0, 0, 0, 1, 2, 0]],
  'id': ['0', '1']})

### 转换ner-tags的数据类型

In [313]:
import datasets
from datasets import ClassLabel, Sequence
class_label = ClassLabel(names=['O', 'B-BIAS', 'I-BIAS'])

new_features = grouped_train_dataset.features.copy()
sequence_class_label = Sequence(feature=class_label, length=-1)
new_features['ner_tags'] = sequence_class_label
grouped_train_dataset_ = grouped_train_dataset.cast(new_features)
grouped_train_dataset_

Casting the dataset:   0%|          | 0/2715 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags', 'id'],
    num_rows: 2715
})

In [316]:
grouped_train_dataset_.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-BIAS', 'I-BIAS'], id=None), length=-1, id=None),
 'id': Value(dtype='string', id=None)}

### 交换key的顺序

In [317]:
def reorder_features(example):
    return {'id': example['id'], 'tokens': example['tokens'], 'ner_tags': example['ner_tags']}

# 使用 map 函数应用这个转换，创建一个新的数据集
reordered_dataset = grouped_train_dataset_.map(reorder_features)

Map:   0%|          | 0/2715 [00:00<?, ? examples/s]

In [321]:
from datasets import Dataset, Features, Value, Sequence, ClassLabel

# 获取原始数据集的特征，并按新的顺序重新排列
original_features = grouped_train_dataset_.features
new_order_features = Features({
    'id': original_features['id'],
    'tokens': original_features['tokens'],
    'ner_tags': original_features['ner_tags']
})

# 创建一个新的数据集，其中列的顺序是按照新的特征顺序排列的
reordered_dataset = grouped_train_dataset_.cast(new_order_features)

# 检查新数据集的特征以确认顺序
print(reordered_dataset.features)

Casting the dataset:   0%|          | 0/2715 [00:00<?, ? examples/s]

{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-BIAS', 'I-BIAS'], id=None), length=-1, id=None)}


In [327]:
reordered_dataset.features, wnut["validation"].features

({'id': Value(dtype='string', id=None),
  'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-BIAS', 'I-BIAS'], id=None), length=-1, id=None)},
 {'id': Value(dtype='string', id=None),
  'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)})

### 2.3 进一步分词和labels对齐

In [276]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [334]:
len(reordered_dataset)

2715

### 🚩清除坏数据！

In [362]:
reordered_dataset[314]
# 包含none元素，会导致使用tokenize_and_align_labels方式中的tokenizer的时候报错：
# TypeError: PreTokenizedEncodeInput must be Union[PreTokenizedInputSequence, 
# Tuple[PreTokenizedInputSequence, PreTokenizedInputSequence]]

{'id': '314',
 'tokens': ['Only',
  '3',
  'concerts',
  'quot',
  'HITY',
  None,
  'CZASIE',
  'quot',
  'in',
  'these',
  'summer',
  'holidays'],
 'ner_tags': [1, 2, 1, 0, 1, 2, 2, 0, 0, 0, 1, 2]}

In [363]:
def remove_none_values(example):
    # 获取tokens和ner_tags列表
    tokens = example['tokens']
    ner_tags = example['ner_tags']
    
    # 使用列表推导式移除None值及其对应的ner_tags
    new_tokens, new_ner_tags = zip(*[(token, tag) for token, tag in zip(tokens, ner_tags) if token is not None])
    
    # 更新example
    example['tokens'] = list(new_tokens)
    example['ner_tags'] = list(new_ner_tags)
    return example

# 应用这个函数到整个数据集
cleaned_dataset = reordered_dataset.map(remove_none_values)

Map:   0%|          | 0/2715 [00:00<?, ? examples/s]

In [368]:
cleaned_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 2715
})

In [373]:
final_dataset = cleaned_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2715 [00:00<?, ? examples/s]

In [374]:
final_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2715
})

### 划分训练集和验证集

In [375]:
split_dataset = final_dataset.train_test_split(test_size=0.2)

# 获取训练集和验证集
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

In [376]:
train_dataset, val_dataset

(Dataset({
     features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 2172
 }),
 Dataset({
     features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 543
 }))

### 🚩上传数据集到hugging face库中

In [394]:
from datasets import DatasetDict, load_dataset

# 假设 cleaned_dataset 是您要上传的数据集
dataset_dict = DatasetDict({
    "train": train_dataset ,
    "validation": val_dataset
})

# 推送数据集到您的 Hugging Face 存储库
dataset_dict.push_to_hub("GXLooong/cleaned_BIAS_CONLL")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

## 3.加载model

In [372]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id_to_label, label2id=label_to_id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 4. 训练

### 4.1 评价指标

In [378]:
import numpy as np
from transformers import TrainingArguments, Trainer

# labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [391]:
torch.__version__

'1.13.1'

In [392]:
training_args = TrainingArguments(
    output_dir="bias_detection_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`